In [1]:
from keras.datasets import imdb # Load the data, keeping only 10,000 of the most frequently occuring words
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i-3, '?') for i in train_data[0]])

import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))    # Creates an all zero matrix of shape (len(sequences),10K)
    for i,sequence in enumerate(sequences):
        results[i,sequence] = 1                        # Sets specific indices of results[i] to 1s
    return results

# Vectorize training Data
X_train = vectorize_sequences(train_data)

# Vectorize testing Data
X_test = vectorize_sequences(test_data)

In [2]:
class_names = ['negative','positive']

y_train = np.asarray(train_labels).astype('float32')
y_test  = np.asarray(test_labels).astype('float32')

y_test

from keras import models 
from keras import layers 
model = models.Sequential() 
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu')) 
model.add(layers.Dense(1, activation='sigmoid'))

from keras import optimizers
from keras import losses 
from keras import metrics 
model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001), loss = losses.binary_crossentropy, metrics = [metrics.binary_accuracy])

X_val = X_train[:10000]
partial_X_train = X_train[10000:] # Labels for validation 
y_val = y_train[:10000] 
partial_y_train = y_train[10000:]

history = model.fit(partial_X_train, partial_y_train, epochs=10, batch_size=512, validation_data=(X_val, y_val))


C:\Users\prati\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 224ms/step - binary_accuracy: 0.7061 - loss: 0.6029 - val_binary_accuracy: 0.8628 - val_loss: 0.4065
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - binary_accuracy: 0.8911 - loss: 0.3453 - val_binary_accuracy: 0.8863 - val_loss: 0.3131
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - binary_accuracy: 0.9222 - loss: 0.2502 - val_binary_accuracy: 0.8897 - val_loss: 0.2848
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - binary_accuracy: 0.9365 - loss: 0.1958 - val_binary_accuracy: 0.8896 - val_loss: 0.2764
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - binary_accuracy: 0.9501 - loss: 0.1612 - val_binary_accuracy: 0.8844 - val_loss: 0.2901
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - binary_accuracy: 0.9587 - loss: 0.1332 - val_binary_accuracy: 0.8868 - val_loss: 0.2847
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - binary_accuracy: 0.9705 - loss: 0.1107 - val_binary_accuracy: 0.8844 - val_loss: 0.2974
Epoch 8/10
3

In [3]:
# Making Predictions for testing data np.set_printoptions(suppress=True) 
import numpy as np

result = model.predict(np.expand_dims(X_test[19],axis=0))
print(result,class_names[int(result[0]>0.5)])

get_phrase = input("Enter: ")

vec_words = []

for i in get_phrase.split(" "):
    vec_words.append(word_index[i.lower()])
    
vec_words = vectorize_sequences(vec_words)
result = []
#print(vec_words)
for j in range(len(vec_words)):
    result99 = model.predict(np.expand_dims(vec_words[j],axis=0))
    #print(result99[0],class_names[int(result99[0]>0.5)])
    result.append(class_names[int(result99[0]>0.5)])


if result.count('negative')> result.count('positive'):
    print("NEGATIVE")
else:
    print("POSITIVE")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
[[0.02275313]] negative
Enter: Great movie loved it
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
POSITIVE


# GUI

In [5]:
import tkinter as tk
import numpy as np

# Function to handle button click
def display_name_and_predict():
    # Get the name from the input field
    name = entry.get()   
    
    vec_words = []
    review_result = []

    for i in name.split(" "):
        vec_words.append(word_index[i.lower()])

    vec_words = vectorize_sequences(vec_words)
    result = []
    #print(vec_words)
    for j in range(len(vec_words)):
        result99 = model.predict(np.expand_dims(vec_words[j],axis=0))
        #print(result99[0],class_names[int(result99[0]>0.5)])
        result.append(result99[0][0])
        review_result.append(class_names[int(result99[0]>0.5)])

    mean = sum(result)/len(result)
    print(review_result)
    final = ''
    if review_result.count('negative')> review_result.count('positive'):
        final = "NEGATIVE"
    else:
        final = "POSITIVE"
    
    # Display the name on the label
    label.config(text=f"Comment is {round(mean,3)}% {final}!")

# Create the main window
window = tk.Tk()
window.title("Name Display and Prediction App")

# Create an input field
entry = tk.Entry(window, width=30)
entry.pack(pady=10)

# Create a button
button = tk.Button(window, text="Predict", command=display_name_and_predict)
button.pack(pady=10)

# Create a display label
label = tk.Label(window, text="")
label.pack(pady=10)

# Run the Tkinter event loop
window.mainloop()